In [1]:
import pandas as pd 
import numpy as np 
from scipy import stats
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
import xgboost as xgb
from xgboost import XGBClassifier 
from xgboost import XGBRegressor
from xgboost import plot_importance
import matplotlib.pylab as plt
from matplotlib import pyplot
import mlxtend
import warnings
import os
from sklearn.metrics import r2_score
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
# 1. Find the start of the ball data 
df = pd.DataFrame()
venueMap = dict()
venueOvers = dict()

for filename in os.listdir("./odi_csv_male dhruv sample/"):  
    if(filename == '.DS_Store'):
        continue
    filename = './odi_csv_male dhruv sample/'+filename
    
    
    file = open(filename)
    
    print(filename)
    rows_to_skip = 0
    nrows = 0
    total_rows=0
    venueSeen = False 
    DL = False

    for line in file:
        if "method" in line:
            DL = True
            print("DL")
        if "venue" in line:
            if venueSeen == False:
                infoArray = line.split(',')
                venue = infoArray[2].strip('"')
                venue = venue.strip()
                venueSeen = True
#             print(venue)
        if not line.startswith('ball'):
            rows_to_skip += 1
        else:
            break
    file.seek(0)
    
    if(DL):
        continue

    for line in file:
        if not line.startswith('2',5,6):
            nrows+=1
        total_rows+=1
    
    file.seek(0)
    nrows=nrows-rows_to_skip
    

    # 2. Create the DataFrame
    df1 = pd.read_csv(
        filepath_or_buffer = file,
        skiprows = rows_to_skip,
        names = ["Ball","Innings", "Balls", "Bat Team", "Batsman","Non_Striker","Bowler","BatsmanScore", "Extras", "Catch", "Catcher"],
        usecols = range(1,11),
        nrows=nrows
    )
   
    

    # 3. Modify columns
    over_col = []
    ball_col = []
    for num in df1['Balls']:
        num1, num2 = str(num).split('.')
        over_col.append(num1)
        ball_col.append(num2)
    df1['Over*'] = list(map(int,over_col))
    df1['Ball*'] = list(map(int,ball_col))
    df1.drop('Balls', axis=1, inplace=True)

    df1['Total balls*'] = df1['Over*']*6 + df1['Ball*']

    df1['Runs*'] = df1['BatsmanScore'].astype('int32') + df1['Extras'].astype('int32')
    Runs= list(df1['Runs*'])

    runs=0
    run_sum=[]
    for run in df1['Runs*']:
        runs+=int(run)
        run_sum.append(runs)
    df1['Total Runs*'] = run_sum   

    rr=0
    run_rate =[]
    for i in range(len(Runs)):
        tot_runs= df1['Total Runs*'].iloc[i]
        rr= df1['Total balls*'].iloc[i]
        rr = tot_runs/rr
        rr*=6
        run_rate.append(round(rr,2))
    df1['Run rate*'] = run_rate

    wicket=0
    wickets=[]
    one_wicket=[]
    for out in df1['Catch']:
        if(not pd.isnull(out)):
            wicket+=1
            one_wicket.append(1)
        else:
            one_wicket.append(0)
        wickets.append(wicket)

    df1['Wickets*'] = wickets
    df1['Game ID'] = filename[-11:-4]
    df1['Venue'] = venue
    endOfDF = df1.tail(1)
#     print(endOfDF)

    totalRuns = endOfDF['Total Runs*'].iloc[0]
    df1['TotalRunsInning'] = totalRuns
    df1['FirstInningScore'] = -1

    
    if venue in venueMap.keys():
#         print(venue + "in keys")
        venueMap[venue].append(endOfDF['Total Runs*'].iloc[0])
        venueOvers[venue].append(endOfDF['Total balls*'].iloc[0])
    else:
#         print(venue + "not in keys")
        venueMap[venue] = [endOfDF['Total Runs*'].iloc[0]]
        venueOvers[venue]=[endOfDF['Total balls*'].iloc[0]]


    rows_to_skip+=nrows

    
    
    
    
    
    # 1. Find the start of the ball data
    # filename = 'CricSheet/odi_csv_male/225171.csv'
    file = open(filename)

    # 2. Create the DataFrame
    df2 = pd.read_csv(
        filepath_or_buffer = file,
        skiprows = rows_to_skip,
        names = ["Ball","Innings", "Balls", "Bat Team", "Batsman","Non_Striker","Bowler","BatsmanScore", "Extras", "Catch", "Catcher"],
        usecols = range(1,11),
        nrows=nrows
    )

    if(df2.empty):
        continue

    # 3. Modify columns
    over_col = []
    ball_col = []
    for num in df2['Balls']:
        num1, num2 = str(num).split('.')
        over_col.append(num1)
        ball_col.append(num2)
    df2['Over*'] = list(map(int,over_col))
    df2['Ball*'] = list(map(int,ball_col))
    df2.drop('Balls', axis=1, inplace=True)

    df2['Total balls*'] = df2['Over*']*6 + df2['Ball*']
    

    
#     if()

    df2['Runs*'] = df2['BatsmanScore'].astype('int32') + df2['Extras'].astype('int32')
    Runs= list(df2['Runs*'])

    runs=0
    run_sum=[]
    for run in df2['Runs*']:
        runs+=int(run)
        run_sum.append(runs)
    df2['Total Runs*'] = run_sum   

    rr=0
    run_rate =[]
    for i in range(len(Runs)):
        tot_runs= df2['Total Runs*'].iloc[i]
        rr= df2['Total balls*'].iloc[i]
        rr = tot_runs/rr
        rr*=6
        run_rate.append(round(rr,2))
    df2['Run rate*'] = run_rate

    wicket=0
    wickets=[]
    one_wicket=[]
    for out in df2['Catch']:
        if(not pd.isnull(out)):
            wicket+=1
            one_wicket.append(1)
        else:
            one_wicket.append(0)
        wickets.append(wicket)

    df2['Wickets*'] = wickets
    df2['Game ID'] = filename[-11:-4]
    df2['Venue'] = venue
    endOfDF = df2.tail(1)
    
    
#     if(endOfDF['Total Runs*'].iloc[0]<100 ) and (endOfDF['Wickets*'].iloc[0]<10)  
#     print(endOfDF['Wickets*'].iloc[0])  

    df2['FirstInningScore'] = totalRuns
    
    totalRuns = endOfDF['Total Runs*'].iloc[0]
    df2['TotalRunsInning'] = totalRuns

    
    if venue in venueMap.keys():
#         print(venue)
        venueMap[venue].append(endOfDF['Total Runs*'].iloc[0])
        venueOvers[venue].append(endOfDF['Total balls*'].iloc[0])

    else:
#         print(venue)
        venueMap[venue] = [endOfDF['Total Runs*'].iloc[0]]
        venueOvers[venue]=[endOfDF['Total balls*'].iloc[0]]

        
#     print(df1['Bat Team'].iloc[0])
#     print(df2['Bat Team'].iloc[0])

    if (((df1['Bat Team'].iloc[0])== "England")or((df1['Bat Team'].iloc[0])== "Afghanistan")or((df1['Bat Team'].iloc[0])== "Pakistan")or((df1['Bat Team'].iloc[0])== "India")or((df1['Bat Team'].iloc[0])== "West Indies")or((df1['Bat Team'].iloc[0])== "Austrailia")or((df1['Bat Team'].iloc[0])== "Bangladesh")or((df1['Bat Team'].iloc[0])== "South Africa")or((df1['Bat Team'].iloc[0])== "New Zealand")or((df1['Bat Team'].iloc[0])== "Sri Lanka")) and (((df2['Bat Team'].iloc[0])== "England")or((df2['Bat Team'].iloc[0])== "Afghanistan")or((df2['Bat Team'].iloc[0])== "Pakistan")or((df2['Bat Team'].iloc[0])== "India")or((df2['Bat Team'].iloc[0])== "West Indies")or((df2['Bat Team'].iloc[0])== "Austrailia")or((df2['Bat Team'].iloc[0])== "Bangladesh")or((df2['Bat Team'].iloc[0])== "South Africa")or((df2['Bat Team'].iloc[0])== "New Zealand")or((df2['Bat Team'].iloc[0])== "Sri Lanka"))  :
#         print("Got in")
        if df.empty:
            frames = [df1,df2]

        else:
            frames = [df1,df2,df]
        df = pd.concat(frames)


In [ ]:
avgVenueMap = dict()
avgList = []

for venue in venueMap.keys():
    avg = sum(venueMap[venue])/sum(venueOvers[venue])
    avgVenueMap[venue] = avg*300
#     print(avg)
#     print(venueMap[venue])
    

for index, row in df.iterrows(): 
#     print(index)
#     print(row["Venue"])
    venue = row["Venue"]
    avgList.append(avgVenueMap[venue])

df["VenueAvg"] = avgList
    


cols = [
 'Innings',
 'Bat Team',
 'Total Runs*',
 'Run rate*',
 'Wickets*',
 'Total balls*',
    'VenueAvg',
    'TotalRunsInning',
    'FirstInningScore'
]
df = df[cols]





In [ ]:
df.to_csv('Sample.csv')

In [4]:
data = pd.read_csv('./Sampletest.csv')
data.head()

,Unnamed: 0,Innings,Bat Team,Total Runs*,Run rate*,Wickets*,Total balls*,VenueAvg,TotalRunsInning,FirstInningScore
0,0,1,England,0,0.0,1,1,273.604061,271,-1
1,1,1,England,1,3.0,1,2,273.604061,271,-1
2,2,1,England,1,2.0,1,3,273.604061,271,-1
3,3,1,England,5,7.5,1,4,273.604061,271,-1
4,4,1,England,6,7.2,1,5,273.604061,271,-1


In [ ]:
def my_scorer(clf, X, y_true):
    
    cols = [
 'Innings',
#  'Bat Team',
 'Total Runs*',
 'Run rate*',
 'Wickets*',
 'Total balls*',
    'VenueAvg',
#     'TotalRunsInning',
    'FirstInningScore'
]
    
    y_pred = clf.predict(X[cols])
    if(X['FirstInningScore'] ==-1):
        error = r2_score(y_true, y_pred)
    else:
        if(y_pred>X['FirstInningScore']):
            error =0
        else:
            error = r2_score(y_true, y_pred)
    return error

In [5]:
feature_cols = [
 'Innings',
#  'Bat Team',
 'Total Runs*',
 'Run rate*',
 'Wickets*',
 'Total balls*',
    'VenueAvg',
#     'TotalRunsInning',
    'FirstInningScore'
]
target_cols =['TotalRunsInning']

features_X = data[feature_cols]
Y = data[target_cols]

In [ ]:
gs = GridSearchCV(estimator=XGBRegressor(
    learning_rate =0.01, 
    n_estimators=2000, 
    max_depth=15,
    min_child_weight=0, 
    gamma=0.01, 
    subsample=0.82, 
    colsample_bytree=0.22,
#     objective= 'reg:squarederror', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
#     reg_alpha = 0, 
#     reg_lambda = 1
),
                  param_grid=[{'n_estimators': [2000]}],
                  cv=5
#                   ,scoring=my_scorer
                 )



gs.fit(features_X,Y)

gs.cv_results_, gs.best_params_, gs.best_score_

In [ ]:
feature_cols = ['BatsmanScore','Over*', 'Runs*', 'Wickets*', 'VenueAvg', 'Total balls*', 'Innings']
target_cols =['Total Runs*']

features_X = df[feature_cols]
Y = df[target_cols]




In [ ]:
param_test1 = {
    'max_depth':range(15,21,1),
    'min_child_weight':range(0,3,1)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( 
    learning_rate =0.09, 
    n_estimators=300, 
    max_depth=32,
    min_child_weight=0,
    gamma=0.0, 
    subsample=0.8, 
    colsample_bytree=0.15,
#     objective= 'binary:logistic', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_lambda=0
), 
param_grid = param_test1, scoring='roc_auc',n_jobs=-1,iid=False, cv=3
                       )
gsearch1.fit(features_X,Y)
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test2 = {
    #gamma':[i/100 for i in range(0,6,1)]
#     'gamma':[0.01,0.02,0,0.03,0.001,0.0001,0.00001]
    'gamma':[.0001,.0002,.00015,.0003,0,0.1,0.02,1]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( 
    learning_rate =0.09, 
    n_estimators=300, 
    max_depth=20,
    min_child_weight=1,
    gamma=0.0, 
    subsample=0.8, 
    colsample_bytree=0.15,
    objective= 'binary:logistic', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_alpha=0,
    reg_lambda = 0

), 
param_grid = param_test2, scoring='roc_auc',n_jobs=-1,iid=False, cv=10)
gsearch1.fit(dataset[predictors],dataset['Y'])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test3 = {
    'subsample':[i/1000.0 for i in range(820,826,1)],
    'colsample_bytree':[i/1000.0 for i in range(220,226,1)]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier(
    learning_rate =0.09, 
    n_estimators=300, 
    max_depth=29,
    min_child_weight=0, 
    gamma=0.01, 
    subsample=0.82, 
    colsample_bytree=0.22,
    objective= 'binary:logistic', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=21,
    reg_alpha = 0, 
    reg_lambda = 1
), 
param_grid = param_test3, scoring='roc_auc',n_jobs=-1,iid=False, cv=10)
gsearch1.fit(dataset[predictors],dataset['Y'])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test4 = {
    'reg_alpha':[0,0.1,1,0.01]
    #'reg_alpha':[0.1,0.2,0,1,2]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( 
    learning_rate =0.09, 
    n_estimators=300, 
    max_depth=20,
    min_child_weight=1,
    gamma=0.0, 
    subsample=0.8, 
    colsample_bytree=0.15,
    objective= 'binary:logistic', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_lambda=0
    
), 
param_grid = param_test4, scoring='roc_auc',n_jobs=-1,iid=False, cv=10)
gsearch1.fit(dataset[predictors],dataset['Y'])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [1]:
model = XGBRegressor(
    learning_rate =0.01, 
    n_estimators=1000, 
    max_depth=4,
    min_child_weight=2, 
    gamma=0.01, 
    subsample=0.81, 
    colsample_bytree=0.22,
    objective= 'reg:squarederror', 
    nthread=16, 
    scale_pos_weight=1, 
    seed=42,
    reg_alpha = 0, 
    reg_lambda = 1
)

kfold = KFold(n_splits=3, random_state=42)
results = cross_val_score(model, features_X, Y, cv=kfold)
print(results)
print(results.mean())
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

NameError: name 'XGBRegressor' is not defined